In [3]:
import networkx as nx
import pandas as pd

def min_cost_delegation(tree, voting_costs, delegating_costs, max_path_length):
    n = len(tree)  # Number of vertices

    # Find root: vertex with no outgoing edges
    is_child = [False] * n
    for i in range(n):
        for child in tree[i]:
            is_child[child] = True

    root = -1
    for i in range(n):
        if not is_child[i]:
            root = i
            break

    if root == -1:
        raise ValueError("No root found. The input is not a valid tree.")

    # Initialize DP table,dp[v][k] = (cost, selected vertices) for subtree rooted at v with max path length k
    dp = [[(float('inf'), []) for _ in range(max_path_length + 1)] for _ in range(n)]

    # bottom-up traversal
    def dfs(vertex):
        children = tree[vertex]

        # Base case: leaf node
        if not children:
            # Leaf vertex votes
            dp[vertex][0] = (voting_costs[vertex], [vertex])

            # Leaf vertex delegates (only valid if max_path_length > 0)
            for k in range(1, max_path_length + 1):
                dp[vertex][k] = (delegating_costs[vertex], [])

            return

        # Process children first
        for child in children:
            dfs(child)

        # Case 1: vertex votes
        selected_vertices = [vertex]
        total_cost = voting_costs[vertex]

        for child in children:
            # For each child, find the best option (vote or delegate with any valid path length)
            min_child_cost = float('inf')
            best_child_selected = []

            # Child votes
            child_cost, child_selected = dp[child][0]
            if child_cost < min_child_cost:
                min_child_cost = child_cost
                best_child_selected = child_selected

            # Child delegates
            for k in range(1, max_path_length + 1):
                child_cost, child_selected = dp[child][k]
                if child_cost < min_child_cost:
                    min_child_cost = child_cost
                    best_child_selected = child_selected

            total_cost += min_child_cost
            selected_vertices.extend(best_child_selected)

        dp[vertex][0] = (total_cost, selected_vertices)

        # Case 2: vertex delegates (only valid if k > 0)
        for k in range(1, max_path_length + 1):
            selected_vertices = []
            total_cost = delegating_costs[vertex]

            for child in children:
                # For each child, find the best option
                # Either child votes, or delegates with path length k-1
                min_child_cost = float('inf')
                best_child_selected = []

                # Child votes
                child_cost, child_selected = dp[child][0]
                if child_cost < min_child_cost:
                    min_child_cost = child_cost
                    best_child_selected = child_selected

                # Child delegates with path length k-1 (only if k > 1)
                if k > 1:
                    child_cost, child_selected = dp[child][k-1]
                    if child_cost < min_child_cost:
                        min_child_cost = child_cost
                        best_child_selected = child_selected

                total_cost += min_child_cost
                selected_vertices.extend(best_child_selected)

            dp[vertex][k] = (total_cost, selected_vertices)

    dfs(root)

    # Root must vote
    min_cost, selected_vertices = dp[root][0]


    print(f"Selected vertices: {selected_vertices}")
    # all_vertices = set(range(len(tree)))
    # delegating_vertices = all_vertices - set(selected_vertices)
    # print(f"Delegating vertices: {list(delegating_vertices)}")
    # total_delegating_cost = sum(delegating_costs[v] for v in delegating_vertices)
    # print(f"Total cost breakdown: Voting = {total_voting_cost}, Delegating = {total_delegating_cost}, Total = {total_voting_cost + total_delegating_cost}")


    total_voting_cost = sum(voting_costs[v] for v in selected_vertices)
    print(f"Total voting cost: {total_voting_cost}")


    return total_voting_cost, selected_vertices

def adjacency_list_to_edgelist(tree):
    """Converts an adjacency list representation to an edge list."""
    edges = []
    for child, parents in enumerate(tree):
        for parent in parents:
            edges.append((parent, child))
    return edges

def is_connected_after_removal(graph, edge):
    """Checks if the graph remains connected after removing the given edge."""
    G = nx.DiGraph(graph)
    G.remove_edge(*edge)
    return nx.is_weakly_connected(G)

def get_tree_after_removal(graph, edge):
    """Returns the tree representation after removing the given edge."""
    G = nx.DiGraph(graph)
    G.remove_edge(*edge)
    tree = [[] for _ in range(len(graph))]
    for u, v in G.edges:
        tree[v].append(u)
    return tree

def min_cost_after_edge_removal(tree, voting_costs, delegating_costs, max_path_length):
    edges = adjacency_list_to_edgelist(tree)
    G = nx.DiGraph(edges)
    min_cost = float('inf')
    best_set = set()

    # If already a tree, run min_cost_delegation without removing anything
    if nx.is_tree(G):
        min_cost, best_set = min_cost_delegation(tree, voting_costs, delegating_costs, max_path_length)

    for edge in G.edges:
        print('remove:',edge)
        if is_connected_after_removal(G, edge):
            new_tree = get_tree_after_removal(G, edge)
            cost, selected_vertices = min_cost_delegation(new_tree, voting_costs, delegating_costs, max_path_length)
            if cost < min_cost:
                min_cost = cost
                best_set = selected_vertices

    return min_cost, best_set

def dfs(graph, node, visited, component):
    visited[node] = True
    component.append(node)
    for neighbor in graph[node]:
        if not visited[neighbor]:
            dfs(graph, neighbor, visited, component)

def run_for_connected_components(tree, voting_costs, delegating_costs, max_path_length):
    # Step 1: Check for connected components
    visited = [False] * len(tree)
    components = []
    final_cost=0
    for node in range(len(tree)):
        if not visited[node]:
            component = []
            dfs(tree, node, visited, component)
            components.append(component)

    # Step 2: Run min_cost_after_edge_removal for each component
    for component in components:
        print()
        # Step 2a: Create an adjacency list for the component using original node IDs
        subgraph = [[] for _ in range(len(tree))]

        # Copy the relevant edges for the current component, preserving original node IDs
        for node in component:
            for neighbor in tree[node]:
                if neighbor in component:  # Only include neighbors within the component
                    subgraph[node].append(neighbor)

        # Step 2b: Call min_cost_after_edge_removal for the subgraph
        min_cost, selected_vertices = min_cost_after_edge_removal(subgraph, voting_costs, delegating_costs, max_path_length)
        final_cost+=min_cost
        # Step 2c: Map selected local component vertices to their original node IDs
        original_selected_vertices = [component[node] for node in selected_vertices]

        # Step 2d: Print the original node IDs (component nodes)
        print(f"Component min_cost: {min_cost}, selected_vertices (original IDs): {original_selected_vertices}")
    print('\nthe total voting cost to be paid is')
    return final_cost


In [4]:
graph = [
    [1],  # Component 1
    [2],  # this is a path of length 2
    [],   # Leaf in Component 1
    [4],  # Component 2; path of length 1
    [],   # Leaf in Component 2
    [6],  # Component 3; path of length 1
    []    # Leaf in Component 3
]
voting_costs = [1, 2, 3, 4, 5, 6, 7]
max_path_length = 1

run_for_connected_components(graph, voting_costs, [1]*len(graph), max_path_length)


Selected vertices: [0, 1]
Total voting cost: 3
remove: (1, 0)
remove: (2, 1)
Component min_cost: 3, selected_vertices (original IDs): [0, 1]

Selected vertices: [0]
Total voting cost: 1
remove: (4, 3)
Component min_cost: 1, selected_vertices (original IDs): [3]

Selected vertices: [0]
Total voting cost: 1
remove: (6, 5)
Component min_cost: 1, selected_vertices (original IDs): [5]

the total voting cost to be paid is


5

In [8]:
df = pd.read_csv('data.csv')

In [40]:
essai = df.loc[(df.CodeQuestion == 'K1')&(df.ExpNum == 'E13')]

In [41]:
graph = []
for i, r in essai.iterrows():
    leaves = list(essai.loc[essai.DelID == r.ID].ID)
    graph.append(leaves)

In [42]:
run_for_connected_components(graph, list(essai.AverageExpertise), [1]*len(graph), 1)


Selected vertices: [0, 16, 33]
Total voting cost: 2.125
remove: (16, 0)
remove: (33, 16)


IndexError: list index out of range

In [29]:
len(graph), len(list(essai.AverageExpertise)), len([1]*len(graph))

(50, 50, 50)